# RNAscope 数据汇总处理程序

本notebook用于处理RNAscope数据，将每个sam_name的多个marker文件合并成一个汇总文件。

主要功能：
1. 读取sample_info.csv获取sam_name列表
2. 为每个sam_name合并对应的summary文件
3. 添加marker前缀并处理列名
4. 计算hasPositive列
5. 保存合并后的汇总文件

In [1]:
# 导入必要的模块
import os
import pandas as pd
from functools import reduce

print('模块导入完成！')

模块导入完成！


In [2]:
# 设置路径和初始化变量
base_folder = 'yourpathway/vg_space_mapping/wt/'  # 当前工作目录作为根目录
summary_folder = os.path.join(base_folder, "summary")
sample_info_path = os.path.join(summary_folder, "sample_info.csv")

print(f'根目录: {base_folder}')
print(f'汇总文件夹: {summary_folder}')
print(f'样本信息文件: {sample_info_path}')

根目录: yourpathway/vg_space_mapping/wt/
汇总文件夹: yourpathway/vg_space_mapping/wt/summary
样本信息文件: yourpathway/vg_space_mapping/wt/summary\sample_info.csv


In [3]:
# 读取 sample_info.csv 文件获取 samName
try:
    sample_info_df = pd.read_csv(sample_info_path)
    # 获取唯一的 samName 列表
    sam_names = sample_info_df['sam_name'].unique()
    print(f'✅ 成功读取样本信息文件，找到 {len(sam_names)} 个唯一的sam_name')
    print(f'sam_name列表: {list(sam_names)}')
except FileNotFoundError:
    print(f"❌ 错误：未找到文件 {sample_info_path}")
    print("请检查文件路径是否正确")
    sam_names = []
except Exception as e:
    print(f"❌ 读取文件时发生错误: {e}")
    sam_names = []

✅ 成功读取样本信息文件，找到 2 个唯一的sam_name
sam_name列表: ['wt', 'tmie']


In [4]:
# 检查汇总文件夹中的文件
if os.path.exists(summary_folder):
    summary_files = os.listdir(summary_folder)
    csv_files = [f for f in summary_files if f.endswith('.csv')]
    print(f'汇总文件夹中共有 {len(csv_files)} 个CSV文件')
    print('前10个CSV文件:')
    for i, file in enumerate(csv_files[:10]):
        print(f'  {i+1}. {file}')
    if len(csv_files) > 10:
        print(f'  ... 还有 {len(csv_files) - 10} 个文件')
else:
    print(f'❌ 汇总文件夹不存在: {summary_folder}')

汇总文件夹中共有 14 个CSV文件
前10个CSV文件:
  1. sample_info.csv
  2. tmie_summary_e.csv
  3. tmie_summary_b.csv
  4. tmie_summary_d.csv
  5. tmie_summary_c.csv
  6. tmie_summary_a.csv
  7. vg_averageing_labels.csv
  8. vg_size.csv
  9. wt_summary_all.csv
  10. wt_summary_e.csv
  ... 还有 4 个文件


In [5]:
def process_sam_name(sam_name, summary_folder):
    """处理单个sam_name的数据合并"""
    print(f"\n正在处理 sam_name: {sam_name}")
    
    # 获取当前samName对应的所有summary文件，排除以_all.csv结尾的文件
    sam_files = [f for f in os.listdir(summary_folder) 
                  if f.startswith(f"{sam_name}_summary_") and not f.endswith("_all.csv")]
    
    if not sam_files:
        print(f"⚠️ 警告：未找到sam_name {sam_name} 对应的文件")
        return False
    
    print(f"找到 {len(sam_files)} 个相关文件:")
    for file in sam_files:
        print(f"  - {file}")
    
    # 初始化一个列表来存储所有数据框
    dataframes = []
    
    # 读取每个文件
    for file in sam_files:
        file_path = os.path.join(summary_folder, file)
        try:
            # 从文件名中提取marker名称
            parts = file.split('_')
            if len(parts) >= 3:
                marker = parts[2].replace('.csv', '')
            else:
                print(f"⚠️ 警告：文件名格式不符合预期 {file}，跳过提取marker。")
                marker = ""

            # 读取CSV文件
            df = pd.read_csv(file_path)
            
            # 确保数据框中有Label列
            if 'Label' not in df.columns:
                print(f"⚠️ 警告：文件 {file_path} 中没有Label列，跳过。")
                continue
            
            # 为除Label外的所有列添加marker前缀 (如果marker不为空)
            if marker:
                rename_dict = {col: f"{marker}_{col}" for col in df.columns if col != 'Label'}
                df = df.rename(columns=rename_dict)
                print(f"    为 {file} 添加了marker前缀: {marker}")
            
            dataframes.append(df)
            
        except Exception as e:
            print(f"❌ 读取文件 {file_path} 时发生错误: {e}")
            continue
    
    if not dataframes:
        print(f"❌ 没有为 sam_name: {sam_name} 找到有效数据")
        return False
    
    print(f"成功读取 {len(dataframes)} 个数据文件")
    
    # 使用reduce函数将所有数据框按Label列横向合并
    if len(dataframes) > 1:
        final_df = reduce(
            lambda left, right: pd.merge(left, right, on='Label', how='outer'),
            dataframes
        )
        print(f"合并完成，最终数据框形状: {final_df.shape}")
    else:
        final_df = dataframes[0]
        print(f"只有一个数据框，无需合并，形状: {final_df.shape}")
    
    # 区分处理isPositive列和其他列的缺失值
    # 首先获取所有isPositive列
    is_positive_cols = [col for col in final_df.columns if col.endswith('_isPositive')]
    # 其他列（排除Label和isPositive列）
    other_cols = [col for col in final_df.columns if col not in is_positive_cols and col != 'Label']
    
    # 对isPositive列的缺失值填充0
    if is_positive_cols:
        final_df[is_positive_cols] = final_df[is_positive_cols].fillna(-1)
        print(f"已将 {len(is_positive_cols)} 个isPositive列的缺失值填充为-1")
    
    # 对其他列的缺失值填充-1
    if other_cols:
        final_df[other_cols] = final_df[other_cols].fillna(-1)
        print(f"已将 {len(other_cols)} 个其他列的缺失值填充为-1")
    
    # 添加hasPositive列
    if is_positive_cols:
        print(f"找到 {len(is_positive_cols)} 个isPositive列: {is_positive_cols}")
        # 确保参与计算的列是数值类型
        for col in is_positive_cols:
            if col in final_df.columns:
                final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0)  # 这里也改为填充0

        # 只要有一个isPositive列的值为1，hasPositive就为1，否则为0
        final_df['hasPositive'] = final_df[is_positive_cols].apply(
            lambda row: 1 if (row == 1).any() else 0, axis=1
        )
        print("已计算hasPositive列")
    else:
        final_df['hasPositive'] = 0
        print("未找到isPositive列，hasPositive列设为全0")
    
    # 在保存前删除所有完全重复的行
    original_rows = len(final_df)
    final_df = final_df.drop_duplicates()
    removed_rows = original_rows - len(final_df)
    if removed_rows > 0:
        print(f"删除了 {removed_rows} 行重复数据")
    
    # 保存合并后的数据框
    output_file_name = f"{sam_name}_summary_all.csv"
    output_file_path = os.path.join(summary_folder, output_file_name)
    final_df.to_csv(output_file_path, index=False)
    print(f"✅ 汇总表格已保存到 {output_file_path}")
    
    return True

print("处理函数定义完成！")

处理函数定义完成！


In [6]:
# 执行数据处理
if len(sam_names) > 0:
    print("===== 开始处理所有sam_name的数据 =====")
    
    success_count = 0
    for sam_name in sam_names:
        if process_sam_name(sam_name, summary_folder):
            success_count += 1
    
    print(f"\n🎉 所有 sam_name 的数据处理完成！")
    print(f"成功处理: {success_count}/{len(sam_names)} 个sam_name")
else:
    print("❌ 没有找到sam_name，无法进行处理")

===== 开始处理所有sam_name的数据 =====

正在处理 sam_name: wt
找到 5 个相关文件:
  - wt_summary_e.csv
  - wt_summary_b.csv
  - wt_summary_d.csv
  - wt_summary_c.csv
  - wt_summary_a.csv
    为 wt_summary_e.csv 添加了marker前缀: e
    为 wt_summary_b.csv 添加了marker前缀: b
    为 wt_summary_d.csv 添加了marker前缀: d
    为 wt_summary_c.csv 添加了marker前缀: c
    为 wt_summary_a.csv 添加了marker前缀: a
成功读取 5 个数据文件
合并完成，最终数据框形状: (3604, 31)
已将 5 个isPositive列的缺失值填充为-1
已将 25 个其他列的缺失值填充为-1
找到 5 个isPositive列: ['e_isPositive', 'b_isPositive', 'd_isPositive', 'c_isPositive', 'a_isPositive']
已计算hasPositive列
✅ 汇总表格已保存到 yourpathway/vg_space_mapping/wt/summary\wt_summary_all.csv

正在处理 sam_name: tmie
找到 5 个相关文件:
  - tmie_summary_e.csv
  - tmie_summary_b.csv
  - tmie_summary_d.csv
  - tmie_summary_c.csv
  - tmie_summary_a.csv
    为 tmie_summary_e.csv 添加了marker前缀: e
    为 tmie_summary_b.csv 添加了marker前缀: b
    为 tmie_summary_d.csv 添加了marker前缀: d
    为 tmie_summary_c.csv 添加了marker前缀: c
    为 tmie_summary_a.csv 添加了marker前缀: a
成功读取 5 个数据文件
合并完成，最终数据框形状

In [7]:
# 检查生成的文件
print("\n===== 检查生成的文件 =====")
if os.path.exists(summary_folder):
    all_files = [f for f in os.listdir(summary_folder) if f.endswith('_all.csv')]
    if all_files:
        print(f"找到 {len(all_files)} 个汇总文件:")
        for file in all_files:
            file_path = os.path.join(summary_folder, file)
            file_size = os.path.getsize(file_path)
            print(f"  - {file} ({file_size} bytes)")
            
            # 显示文件内容预览
            try:
                df = pd.read_csv(file_path)
                print(f"    形状: {df.shape}, 列数: {len(df.columns)}")
                print(f"    列名: {list(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
            except Exception as e:
                print(f"    无法读取文件内容: {e}")
    else:
        print("未找到任何汇总文件")
else:
    print(f"汇总文件夹不存在: {summary_folder}")


===== 检查生成的文件 =====
找到 2 个汇总文件:
  - tmie_summary_all.csv (933631 bytes)
    形状: (2090, 32), 列数: 32
    列名: ['Label', 'e_pos_X', 'e_pos_Y', 'e_warpedROIvar1', 'e_warpedROIvar2']...
  - wt_summary_all.csv (1581245 bytes)
    形状: (3604, 32), 列数: 32
    列名: ['Label', 'e_pos_X', 'e_pos_Y', 'e_warpedROIvar1', 'e_warpedROIvar2']...


## 📊 数据处理完成！

所有sam_name的数据已成功处理并合并。

**处理结果**:
- 每个sam_name生成了一个 `{sam_name}_summary_all.csv` 文件
- 所有marker数据按Label列合并
- 添加了hasPositive列标识阳性结果
- 重复数据已自动清理

**下一步**: 可以使用生成的文件进行后续分析或可视化。